In [1]:
from gssutils import *

scraper = Scraper('https://www.ons.gov.uk/businessindustryandtrade/business/businessinnovation/datasets/' \
                  'foreigndirectinvestmentinvolvingukcompaniesoutwardtables')
scraper

## Foreign direct investment involving UK companies: outward

Annual statistics on the investment of UK companies abroad, including for investment flows, positions and earnings.

### Distributions

1. Foreign direct investment involving UK companies: outward ([MS Excel Spreadsheet](https://www.ons.gov.uk/file?uri=/businessindustryandtrade/business/businessinnovation/datasets/foreigndirectinvestmentinvolvingukcompaniesoutwardtables/current/annualforeigndirectinvestment2017outward.xls))


In [2]:
sheets = {sheet.name: sheet for sheet in scraper.distribution().as_databaker()}
sheets.keys()

dict_keys(['Contents', '1.1', '1.2', '1.3', '2.1', '2.2', '2.3', '3.1', '3.2 ', '3.3', '4.1 ', '4.2 ', '4.3', 'Geography', 'SIC'])

In [3]:
sh = ['2.2','3.2 ','4.2 ']

In [4]:
table = pd.DataFrame()

In [5]:
for sh_id in sh:
    by_area = sheets[sh_id]
    years = by_area.excel_ref('D6').expand(DOWN).is_not_blank().is_not_whitespace()
    areas = by_area.excel_ref('A5').expand(DOWN).is_not_blank()
    sub_areas = by_area.excel_ref('B5').expand(DOWN).is_not_blank().filter(lambda c: c.value.strip() != 'of which')
    countries = by_area.excel_ref('C').expand(DOWN).by_index([7,11,15,19,23,27,31,35,39,43,47,51,55,59,63,67,71,75,79,83,87,
                                                            91,95,99,103,107,111,115,119,123,127,131,135,139,143,147,151,155,159,
                                                             163,167,171,175,179,183,187,191,195,199,203,207,211,215,219,223,227])
    components = by_area.excel_ref('E5:M6').expand(RIGHT).is_not_blank().is_not_whitespace()
    components = components - by_area.excel_ref('E5') - by_area.excel_ref('H5') - by_area.excel_ref('L5')                     
    obs = by_area.excel_ref('E7').expand(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace() 
    cs = ConversionSegment(
        obs, [
            HDim(years, 'Year', DIRECTLY, LEFT),
            HDim(areas, 'Area', CLOSEST,ABOVE),
            HDim(sub_areas, 'Sub area', CLOSEST,ABOVE),
            HDim(countries, 'Country', CLOSEST,ABOVE),
            HDimConst('Investment Direction', 'outward'),
            HDim(components,'Component', DIRECTLY, ABOVE),
            HDimConst('Industry', 'All activities'),
            HDimConst('International Trade Basis', 'BPM5')
        ]
    )
    # savepreviewhtml(cs)
    t_by_area = cs.topandas()    
    
    def user_perc(x,y,z):

        if x.strip() == '':
            if (y == '') | (y == None) | (y == ' '):
                return z
            else :
                return y
        else:
            return x

    t_by_area['ONS FDI Area'] = 'fdi/' + t_by_area.apply(lambda row: user_perc(row['Country'],row['Sub area'],row['Area']), axis = 1)
    t_by_area['ONS FDI Area'] = t_by_area['ONS FDI Area'].str.strip()
    t_by_area.drop(columns=['DATAMARKER','Area', 'Sub area','Country'], inplace=True)
    t_by_area.rename(columns={'OBS': 'Value'}, inplace=True)

    t_by_area.replace('', pd.np.nan, inplace=True)
    t_by_area.dropna(subset=['Value'], inplace=True)

    t_by_area[['Value', 'Year']] = t_by_area[['Value', 'Year']].astype(float).astype(int)
    t_by_area['Component'].fillna('Total net FDI earnings abroad', inplace = True)

    table = pd.concat([table, t_by_area])

In [6]:
sr = ['2.3','3.3','4.3']

In [7]:
fc = ['Total net foreign direct investment in the UK',
      'Total net FDI international investment position in the UK at end period',
      'Total net FDI earnings in the UK']

In [8]:
i = 0
for sh_id in sr:
    by_area = sheets[sh_id]
    years = by_area.excel_ref('E6').expand(DOWN).is_not_blank().is_not_whitespace()
    areas = by_area.excel_ref('A6').expand(DOWN).is_not_blank()
    sub_areas = by_area.excel_ref('B6').expand(DOWN).is_not_blank().filter(lambda c: c.value.strip() != 'of which')
    countries = by_area.excel_ref('C').expand(DOWN).by_index([7,11,15,19,23,27,31,35,39,43,47,51,55,59,63,67,71,75,79,83,87,
                                                            91,95,99,103,107,111,115,119,123,127,131,135,139,143,147,151,155,159,
                                                             163,167,171,175,179,183,187,191,195,199,203,207,211,215,219,223,227])
    components = by_area.excel_ref('F5').expand(RIGHT).is_not_blank().is_not_whitespace()
    obs = by_area.excel_ref('F6').expand(DOWN).expand(RIGHT).is_not_blank().is_not_whitespace()
    cs = ConversionSegment(
        obs, [
            HDim(years, 'Year', DIRECTLY, LEFT),
            HDim(areas, 'Area', CLOSEST,ABOVE),
            HDim(sub_areas, 'Sub area', CLOSEST,ABOVE),
            HDim(countries, 'Country', CLOSEST,ABOVE),
            HDimConst('Investment Direction', 'outward'),
            HDim(components,'Industry', DIRECTLY, ABOVE),
            HDimConst('International Trade Basis', 'BPM5')
        ]
    )
    # savepreviewhtml(cs)
    t_by_area = cs.topandas()


    def user_perc(x,y,z):

        if (x == '') | (x == ' ') | (x == None) :
            if (y == '') | (y == None) | (y == ' '):
                return z
            else :
                return y
        else:
            return x

    t_by_area['ONS FDI Area'] = 'fdi/' + t_by_area.apply(lambda row: user_perc(row['Country'],row['Sub area'],row['Area']), axis = 1)
    t_by_area['ONS FDI Area'] = t_by_area['ONS FDI Area'].str.strip()
    t_by_area.drop(columns=['DATAMARKER','Area', 'Sub area','Country'], inplace=True)
    t_by_area.rename(columns={'OBS': 'Value'}, inplace=True)

    t_by_area.replace('', pd.np.nan, inplace=True)
    t_by_area.dropna(subset=['Value'], inplace=True)

    t_by_area[['Value', 'Year']] = t_by_area[['Value', 'Year']].astype(float).astype(int)
    t_by_area['Component'] = fc[i]
    i = i + 1

    table = pd.concat([table, t_by_area])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [9]:
st = ['2.1','3.1','4.1 ']

In [10]:
fc = ['Total net foreign direct investment in the UK',
      'Total net FDI international investment position in the UK at end period',
      'Total net FDI earnings in the UK']

In [11]:
i = 0
for sh_id in st:
    by_area = sheets[sh_id]
    years = by_area.excel_ref('A5').expand(RIGHT).is_not_blank().is_not_whitespace()
    areas = by_area.excel_ref('A5').expand(DOWN).is_not_blank()
    sub_areas = by_area.excel_ref('B5').expand(DOWN).is_not_blank().filter(lambda c: c.value.strip() != 'of which')
    countries = by_area.excel_ref('C5').expand(DOWN).is_not_blank().is_not_whitespace()
    obs = years.fill(DOWN)
    obs = obs & (sub_areas | areas | countries).expand(RIGHT)
    cs = ConversionSegment(
        obs, [
            HDim(years, 'Year', DIRECTLY, ABOVE),
            HDim(areas, 'Area', DIRECTLY, LEFT),
            HDim(sub_areas, 'Sub area', DIRECTLY, LEFT),
            HDim(countries, 'Country', DIRECTLY, LEFT),
            HDimConst('Investment Direction', 'outward'),
            HDimConst('Industry', 'All activities'),
            HDimConst('International Trade Basis', 'BPM5')
        ]
    )
    # savepreviewhtml(cs)
    t_by_area = cs.topandas()
    t_by_area['ONS FDI Area'] = t_by_area.apply(
        lambda row: ''.join(
            [s.strip() for s in [row['Area'], row['Sub area'], row['Country']] if s != None]), axis=1)
    t_by_area['ONS FDI Area'] = 'fdi/' + t_by_area['ONS FDI Area']
    t_by_area.drop(columns=['DATAMARKER','Area', 'Sub area','Country'], inplace=True)
    t_by_area.rename(columns={'OBS': 'Value'}, inplace=True)

    t_by_area.replace('', pd.np.nan, inplace=True)
    t_by_area.dropna(subset=['Value'], inplace=True)

    t_by_area[['Value', 'Year']] = t_by_area[['Value', 'Year']].astype(float).astype(int)
    t_by_area['Component'] = fc[i]
    i = i + 1

    table = pd.concat([table, t_by_area])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:37: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [12]:
table['International Trade Basis'] = ['BPM6' if x>=2012 else 'BPM5' for x in table['Year']]

In [13]:
table['ONS FDI Area'] = table['ONS FDI Area'].str.lower()

In [14]:
table['Industry'] = table['Industry'].map(
    lambda x: {
        'Total' : 'All activities'
        }.get(x, x))

In [15]:
table['Component'] = table['Component'].map(
    lambda x: {
        "Foreign companies' share of UK companies' net profits" : "Foreign parent companies' share of UK companies' net profits"
        }.get(x, x))

In [16]:
table['Industry'] = table['Industry'].map(
    lambda x: {
        'Agriculture, forest & fishing' : 'Agriculture, forestry & fishing',
        'Retails & wholesale trade, repair of motor vehicles & motor cycles,' : 'Retail & wholesale trade, repair of motor vehicles & motor cycles'
        }.get(x, x))

In [17]:
import urllib.request as request
import csv
import io
import requests
r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/fdi-component.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
url="https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/fdi-component.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
table = pd.merge(table, c, how = 'left', left_on = 'Component', right_on = 'Label')
table.columns = ['FDI Component' if x=='Notation' else x for x in table.columns]
r = request.urlopen('https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/sic-industry.csv').read().decode('utf8').split("\n")
reader = csv.reader(r)
url="https://raw.githubusercontent.com/ONS-OpenData/ref_trade/master/codelists/sic-industry.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
table = pd.merge(table, c, how = 'left', left_on = 'Industry', right_on = 'Label')
table.columns = ['SIC Industry' if x=='Notation' else x for x in table.columns]

In [18]:
table['ONS FDI Area'] = table['ONS FDI Area'].map(
    lambda x: {
        'fdi/other european' : 'fdi/other-european-countries',
        'fdi/countries' : 'fdi/other-european-countries',
        'fdi/uk offshore' : 'fdi/uk-offshore-islands',
        'fdi/near & middle east' : 'fdi/near-and-middle-east-countries',
        'fdi/other asian' : 'fdi/other-asian-countries',
        'fdi/central &' : 'fdi/other-asian-countries',
        'fdi/eastern' : 'fdi/other-asian-countries',
        'fdi/other' : 'fdi/other-european-countries',
        'fdi/european' : 'fdi/other-european-countries',
        'fdi/islands' : 'fdi/uk-offshore-islands',
        'fdi/near & middle' : 'fdi/near-and-middle-east-countries',
        'fdi/east countries' : 'fdi/near-and-middle-east-countries',
        'fdi/gulf arabian' : 'fdi/gulf-arabian-countries',
        'fdi/south korea'  : 'fdi/south-korea',
        'fdi/hong kong'  : 'fdi/hong-kong',
        'fdi/south africa' : 'fdi/south-africa',
        'fdi/the americas'  : 'fdi/the-americas',
        'fdi/australasia & oceania' : 'fdi/australasia-and-oceania',
        'fdi/world total' : 'fdi/world-total',
        'fdi/central & eastern europe' : 'fdi/central-and-eastern-europe',
        'fdi/new zealand' : 'fdi/new-zealand',
        'fdi/other european countries' :  'fdi/other-european-countries',
        'fdi/uk offshore islands' : 'fdi/uk-offshore-islands', 
        'fdi/near & middle east countries'  : 'fdi/near-and-middle-east-countries',
        'fdi/other asian countries' : 'fdi/other-asian-countries',
        'fdi/gulf arabian countries' : 'fdi/gulf-arabian-countries',
        'fdi/czech republic' : 'fdi/czech-republic',
        'fdi/irish republic' : 'fdi/irish-republic'       
        }.get(x, x))

In [19]:
table.drop_duplicates(inplace = True)

In [20]:
table.drop(columns=['Component','Industry', 'Label_x','Parent Notation_x', 'Sort Priority_x',
                   'Description_x', 'Label_y','Parent Notation_y', 'Sort Priority_y',
                   'Description_y' ], inplace=True)

In [21]:
table['Unit'] = 'gbp-million'
table['Measure Type'] = 'GBP Total'